In [1]:
import sys
sys.path.append('../scripts')

In [2]:
import pandas as pd
import openai
from pathlib import Path
from evaluation import error_analysis, get_scores
import re

In [3]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path='..', version_base='1.3.1')
config = compose(config_name='experiment.yaml')

In [4]:
openai.api_key = config.api_key

In [13]:
prompt_to_use = 0
prompts = [
    "Ich werde dir im Folgenden einen Satz zeigen, welcher sogennannte Koordinationsellipsen enthält. Du sollst sie finden und auflösen. Ein Beispiel:'Dieser Unterschied zeigte sich sowohl im Stadium I/II als auch im Stadium III [REF].' Deine Antwort:'Dieser Unterschied zeigte sich sowohl im Stadium I/Stadium II als auch im Stadium III [REF].'. Ein weiteres Beispiel:'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und –ausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Deine Antwort:'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und Tumorausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Ein weiteres Beispiel:'Eine Metaanalyse von 16 randomisierten Studien mit BRAF- oder/und MEK-Inhibitoren bestätigt die Überlegenheit der Kombinationstherapie [REF].' Deine Antwort:'Eine Metaanalyse von 16 randomisierten Studien mit BRAF-Inhibtoren oder/und MEK-Inhibitoren bestätigt die Überlegenheit der Kombinationstherapie [REF].' Ein letztes Beispiel:'Das Erkrankungs- und Sterberisiko ist jedoch deutlich unterschiedlich.' Deine Antwort:'Das Erkrankungsrisiko und Sterberisiko ist jedoch deutlich unterschiedlich.' Wenn ich dir gleich mein Beispiel gebe, antworte nur mit dem korrigierten Satz und keiner Erklärung. Korrigiere bitte folgenden Satz:",
    "Ich werde dir im Folgenden einen Satz zeigen, welcher sogennannte Koordinationsellipsen enthält. Du sollst sie finden und auflösen. Ein Beispiel: 'Dieser Unterschied zeigte sich sowohl im Stadium I/II als auch im Stadium III [REF].' Deine Antwort: 'Dieser Unterschied zeigte sich sowohl im Stadium I/Stadium II als auch im Stadium III [REF].' Ein weiteres Beispiel: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und –ausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Deine Antwort: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und Tumorausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Korrigiere bitte folgenden Satz:",
    "Ich werde dir im Folgenden einen Satz zeigen. Möglicherweise enthält dieser Koordinationsellipsen. Ich möchte, dass du erstmal herausfindest, ob in dem Satz solche Koordinationsellipsen vorkommen. Danach sollst du sie auflösen. Ein Beispiel: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und –ausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Hier kannst du 'Tumormerkmale und -ausdehung' als Koordinationsellipse erkennen. Deine Antwort wäre nun: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und Tumorausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Du sollst also nichts an dem Satz ändern außer die Koordinationsellipsen zu korrigieren. Wenn ich dir gleich mein Beispiel gebe, antworte nur mit dem korrigierten Satz und keiner Erklärung. Mein Beispiel:",
    "Ich werde dir im Folgenden einen Satz zeigen, welcher sogennannte Koordinationsellipsen enthält. Du sollst sie finden und auflösen. Ein Beispiel:'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinien- und Rezidivtherapiee in der CLL zugelassen.' Deine Antwort:'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinientherapie und Rezidivtherapiee in der CLL zugelassen.' Korrigiere bitte folgenden Satz:",
    "In der deutschen Sprache gibt es sogenannte Koordinationsellipsen. Ein Beispiel für eine Koordinationsellipse ist:'Stadium I/II'. Die richtige Auflösung:'Stadium I/Stadium II'. Ein weiteres Beispiel:'Tumormerkmale und –ausdehnung'. Die richtige Auflösung:'Tumormerkmale und Tumorausdehnung'. Ein weiteres Beispiel:'BRAF- oder/und MEK-Inhibitoren'. Die richtige Auflösung:'BRAF-Inhibtoren oder/und MEK-Inhibitoren'. Ein letztes Beispiel:'Erkrankungs-, Verhinderungs- und Sterberisiko' Die richtige Auflösung:'Erkrankungsrisiko und Sterberisiko'. Ich werde dir im Folgenden einen Satz zeigen, welcher eine oder mehrere dieser Koordinationsellipsen enthält. Ich möchte, dass du sie findest und auflöst. Ein Beispiel:'Das Erkrankungs- und Sterberisiko ist jedoch deutlich unterschiedlich [REF].' Deine Antwort:'Das Erkrankungsrisiko und Sterberisiko ist jedoch deutlich unterschiedlich [REF].' Korrigiere bitte folgenden Satz:"
]

In [14]:
ellipses = pd.read_csv(Path('..') / config.data.cnf_tsv_path, sep='\t')
not_ellipses = pd.read_csv(Path('..') / config.data.controls_tsv_path, sep='\t')

not_ellipses['full_resolution'] = not_ellipses.raw_sentence
ellipses = ellipses[ellipses.split == 'test']
not_ellipses = not_ellipses[not_ellipses.split == 'test']

data = pd.concat([ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']], not_ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']]]).sort_values(by='file').reset_index(drop=True)

In [15]:
test_data = data.sample(50, random_state=42)

In [8]:
def get_openai_response(prompt, example):
    return openai.Completion.create(
        model="text-davinci-003",
        prompt= prompt + example,
        temperature=0,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

In [42]:
predictions = []
for example, resolution in zip(test_data.raw_sentence, test_data.full_resolution):
    predictions.append(get_openai_response(prompts[prompt_to_use], example)['choices'][0]["text"])

In [46]:
for i, prediction in enumerate(predictions):
    predictions[i] = prediction.replace('\n', '')
    predictions[i] = re.sub(r'.*Antwort:\s?', '', predictions[i])
    if predictions[i][0] == "'":
        predictions[i] = predictions[i][1:-1]

In [48]:
ea = error_analysis(predictions, test_data.full_resolution, test_data.raw_sentence)
print(get_scores(ea, "eval"))

{'eval/tp': 0.26, 'eval/tp_abs': 13, 'eval/fn': 0.02, 'eval/fn_abs': 1, 'eval/fp': 0.3, 'eval/fp_abs': 15, 'eval/replace': 0.1, 'eval/replace_abs': 5, 'eval/insert': 0.02, 'eval/insert_abs': 1, 'eval/delete': 0.06, 'eval/delete_abs': 3, 'eval/complex': 0.24, 'eval/complex_abs': 12, 'eval/edit_distance_rel': 0.3470888663906745, 'eval/exact_match': 0.26, 'eval/gleu': 0.7530511060259344}


In [47]:
final = pd.DataFrame(data={'resolution': test_data.full_resolution, 'prediction': predictions})
final[final.prediction!=final.resolution]

,resolution,prediction
1089,Die berichteten Nebenwirkungen (alle Grad 1) u...,Die berichteten Nebenwirkungen (alle Grad 1) u...
1103,Demgegenüber konnte eine präoperative oder per...,Demgegenüber konnte eine präoperative oder per...
739,"Aus den wenigen Studien, die nach Anzahl befal...","Aus den wenigen Studien, die nach Anzahl befal..."
140,Das Vorhandensein einzelner ipsilateraler pulm...,Das Vorhandensein einzelner ipsilateraler pulm...
1018,In einer klinischen Interaktionsstudie waren z...,In einer klinischen Interaktionsstudie waren z...
527,Patientinnen mit frühem Ovarialkarzinom und ko...,Patientinnen mit frühem Ovarialkarzinom und ko...
812,Palliativmedizin/Palliativversorgung bejaht da...,"a call to action, European Association for Pa..."
1118,GIST des Magens mit einem Durchmesser kleiner ...,GIST des Magens mit einem Durchmesser kleiner ...
950,Dabei gab es bezüglich der Lebensqualität kein...,Dabei gab es bezüglich der Lebensqualität kein...
420,Aufgrund der Zunahme der abdominellen Schnittb...,Aufgrund der Zunahme der abdominellen Schnittb...
